# 路径还原与问题提交

代码参见 `../src/submit.jl`，调用 `submit(predict)` 将预测数据转化为提交格式

函数说明：
- `standard_triple(triple, lang)` 还原单个带链接的三元组
- `submit_format(triples, langs)` 将多个三元组转化为标准格式
- `standard_path(path)` 将路径中的三元组转化为翻译前的三元组，同时返回语言信息
- `find_paths(ner, rels)` 寻找以 `ner` 为起点，`rels` 为路径的子图
- `submit(sol; check = false)` 根据预测数据检索路径，`check` 指定是否根据跳数及跨语言进行筛选

补充说明：
- `find_paths, submit` 依赖变量 `edges`
- `recover_paths` 依赖变量 `MT_zh_new2raw, MT_en_new2raw`
- 这三个函数与翻译相关

### 问题提交

In [3]:
include("../src/translatedata.jl")
include("../src/submit.jl")
mkpath("submit")

"submit"

In [9]:
# 读取预测结果
sols = split(strip(read(open("predict_data.txt", "r"), String)), '\n')
# sols = [replace(sol, r"#\d+" => "") for sol in sols]

In [7]:
fails, multisol, submit_id = String[], String[], 7
open("submit/submit_$(submit_id).txt", "w") do io
    println(io, "id\tans_path")
    for (i, predict) in enumerate(sols)
        lines = submit(predict; check=true)
        # 多解的情况
        length(lines) > 1 && push!(multisol, predict)
        # 没有合适解的情况
        isempty(lines) && (lines = submit(predict); push!(fails, predict))
        println(io, i-1, '\t', last(lines))
    end
end

In [8]:
fails;

In [11]:
function count_list(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

count_list (generic function with 1 method)